In [1]:
import subprocess, os, time, re, csv, sys, io

import pandas as pd
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from pathlib import Path
from typing import List, Tuple
from collections import defaultdict

from visualization import *
from nibabel.orientations import (
    io_orientation,
    axcodes2ornt,
    ornt_transform,
    apply_orientation,
)

root_dir = "/valiant02/masi/zuol1/data/cibs_brain2/"
dest_root_dir = "/nfs2/harmonization/BIDS/CIBS_BRAIN2_Harmonized"
result_dir = "dataset_cache"

In [2]:
def dump(path_list, outfile):
    with open(outfile, "w") as f:
        for p in sorted(path_list):
            f.write(str(p) + "\n")
    print(f"Cached {len(path_list)} paths to '{outfile}'")


def load_paths_from_file(file_path: str) -> List[str]:
    """Load paths from a file."""
    with open(file_path, "r") as f:
        return [line.strip() for line in f if line.strip()]

In [5]:
lines = []
for dirpath, dirnames, filenames in os.walk(Path(root_dir)):
    if dirnames:
        continue
    file_type = Path(dirpath).name
    session = Path(dirpath).parent.name
    if session not in {"00", "12"}:
        continue
    for fname in filenames:
        if fname.endswith(".nii") or fname.endswith(".nii.gz"):
            lines.append(str(Path(dirpath) / fname))
dump(lines, f"{result_dir}/nifti_all.txt")

Cached 3730 paths to 'dataset_cache/nifti_all.txt'


In [3]:
lines = load_paths_from_file(f"{result_dir}/nifti_nii.txt")
print(f"Loaded {len(lines)} paths from nifti_nii.txt")

Loaded 722 paths from nifti_nii.txt


In [4]:
depths = [len(Path(p).parts) for p in lines]

# Check whether the depths of all addresses are same
print(len(set(depths)) == 1)

True


In [6]:
subject_sessions = defaultdict(set)

for fp in lines:
    p = Path(fp)
    if len(p.parents) < 3:
        continue
    session = p.parents[1].name
    file_type = p.parents[0].name
    if session not in ["00", "12"] or file_type != "nii":
        continue
    subject_dir = p.parents[2]
    subject_sessions[subject_dir].add(session)

only_00 = [p for p, s in subject_sessions.items() if s == {"00"}]
both_00_12 = [p for p, s in subject_sessions.items() if s == {"00", "12"}]
only_12 = [p for p, s in subject_sessions.items() if s == {"12"}]

dump(only_00, f"{result_dir}/subject_only_00.txt")
dump(both_00_12, f"{result_dir}/subject_00_and_12.txt")
dump(only_12, f"{result_dir}/subject_only_12.txt")

print(f"Only baseline data (00 only): {len(only_00)}")
print(f"Valid data (00 + 12): {len(both_00_12)}")
print(f"Only 12-month data: {len(only_12)}")

Cached 76 paths to 'dataset_cache/subject_only_00.txt'
Cached 37 paths to 'dataset_cache/subject_00_and_12.txt'
Cached 22 paths to 'dataset_cache/subject_only_12.txt'
Only baseline data (00 only): 76
Valid data (00 + 12): 37
Only 12-month data: 22


In [ ]:
file_pat = re.compile(r"BRAIN_T1-3D.*_n4_reg_harmonized_harmonized_fusion\.nii\.gz$")

good_paths: list[Path] = []
bad_subjects: list[str] = []
subject_roots = only_00 + both_00_12 + only_12

for subj_root in subject_roots:
    subj_root = Path(subj_root)
    subj_id = subj_root.name

    for ses in ("00", "12"):
        ses_dir = subj_root / ses
        if not ses_dir.is_dir():
            continue
        proc_dir = ses_dir / "proc"
        if not proc_dir.is_dir():
            bad_subjects.append(f"{subj_id}: missing directory {ses}/proc")
            break
        matches = [
            p for p in proc_dir.iterdir() if p.is_file() and file_pat.search(p.name)
        ]
        if not matches:
            bad_subjects.append(
                f"{subj_id}: no BRAIN_T1-3D harmonized file in {ses}/proc"
            )
            break
        good_paths.extend(matches)

dump(good_paths, f"{result_dir}/harmonized_all.txt")

if bad_subjects:
    print("\n Subjects with missing data:")
    for msg in bad_subjects:
        print("  -", msg)
else:
    print("\n All subjects passed the file-check.")

Cached 168 paths to 'dataset_cache/harmonized_all.txt'

 Subjects with missing data:
  - BRA105: no BRAIN_T1-3D harmonized file in 00/proc
  - BRA107: no BRAIN_T1-3D harmonized file in 12/proc
  - BRA132: no BRAIN_T1-3D harmonized file in 12/proc
  - VUM241: no BRAIN_T1-3D harmonized file in 12/proc


In [11]:
good_paths = load_paths_from_file(f"{result_dir}/harmonized_all.txt")
print(f"Loaded {len(good_paths)} paths from harmonized_all.txt")


def extract_modality(fname: str) -> str:
    stem = fname
    for ext in (".nii.gz", ".nii"):
        if stem.endswith(ext):
            stem = stem[: -len(ext)]
            break
    try:
        seq_part = stem.split("_BRAIN_")[1]
    except IndexError:
        return ""
    tokens = seq_part.split("-")
    return "-".join(tokens[:2]) if len(tokens) >= 2 else ""


def parse_path(file_path: str) -> dict | None:
    p = Path(file_path)
    parts = p.parts
    if len(parts) != 10:
        raise RuntimeError(f"File address not valid.\nAddress: {file_path}")
    subject = parts[6]
    session = parts[7]
    scan_type = extract_modality(parts[9])
    if scan_type not in {"T1-3D"}:
        return None
    return {
        "filepath": p.resolve(),
        "scan_type": scan_type,
        "subject_id": f"sub-{subject}",
        "session_id": f"ses-{session}",
    }


rows = [row for fp in good_paths if (row := parse_path(fp))]
df = pd.DataFrame(rows)

Loaded 168 paths from harmonized_all.txt


In [12]:
df["run"] = (
    df.groupby(["scan_type", "subject_id", "session_id"])
    .cumcount()
    .add(1)
    .astype("string")
)

mask = (
    df.groupby(["scan_type", "subject_id", "session_id"])["run"].transform("size").eq(1)
)
df.loc[mask, "run"] = ""

In [13]:
def make_links(row):
    prefix = f"{row.subject_id}_{row.session_id}"
    if row.run:
        prefix += f"_run-{row.run}"
    if row.scan_type == "T1-3D":
        fname = f"{prefix}_T1w"
        subdir = "anat"
    else:
        raise ValueError(f"Unknown scan type: {row.scan_type}")
    base = f"{dest_root_dir}/{row.subject_id}/{row.session_id}/{subdir}/{fname}"
    row["nii_link"] = base + ".nii.gz"
    return row


df = df.apply(make_links, axis=1)

In [14]:
df.sort_values(["scan_type", "subject_id", "session_id", "run"], inplace=True)

In [15]:
mask = (
    (df["scan_type"] == "T1-3D")
    & df["run"].notna()
    & (df["run"] != "")
    # & (df["run"] == "")
)

if mask.any():
    print(df.loc[mask])
else:
    print("Nothing here.")

Nothing here.


In [16]:
df.to_csv(f"{result_dir}/data.tsv", sep="\t")

In [17]:
def load_dataframe(path: str) -> pd.DataFrame:
    if path.endswith(".pkl"):
        return pd.read_pickle(path)
    return pd.read_csv(path, sep=None, engine="python")


def _collect_link_pairs(df: pd.DataFrame) -> List[Tuple[str, str]]:
    colmap = {
        "filepath": "nii_link",
    }

    present_pairs = [
        (src, dst)
        for src, dst in colmap.items()
        if src in df.columns and dst in df.columns
    ]
    if not present_pairs:
        raise ValueError("No recognised source/target column pairs found.")

    commands = []
    for _, row in df.iterrows():
        for src, dst in present_pairs:
            s, d = row[src], row[dst]
            if pd.notna(s) and pd.notna(d) and str(s).strip() and str(d).strip():
                commands.append(f'mkdir -p "{Path(d).parent}"')
                commands.append(f'ln -s "{s}" "{d}"')
    return commands


def write_link_commands(
    df: pd.DataFrame,
    output_txt: str | Path,
    *,
    overwrite: bool = True,
) -> None:
    output_txt = Path(output_txt)
    mode = "w" if overwrite else "a"

    commands = _collect_link_pairs(df)

    with output_txt.open(mode, encoding="utf-8") as f:
        for cmd in commands:
            f.write(cmd + "\n")

    print(f"{len(commands)} link commands written to {output_txt.resolve()}")

In [18]:
df = load_dataframe(f"{result_dir}/data.tsv")

In [19]:
write_link_commands(df, f"{result_dir}/data_link_command.txt")

336 link commands written to /nfs/ForHenry/brain_ventricle/dataset_cache/data_link_command.txt


In [ ]:
def fig_to_ndarray(fig):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=fig.dpi, bbox_inches="tight")
    buf.seek(0)
    img = plt.imread(buf)
    buf.close()
    return img


dpi = 100
pdf_path = Path(result_dir) / "Harmonization_QC.pdf"
with PdfPages(pdf_path) as pdf:
    for file_path in df["filepath"]:
        p = Path(file_path)
        subj = p.parts[6]
        sess = p.parts[7]

        ori_path = (
            Path(*p.parts[:8]) / "nii" / (p.parts[9].split("_n4_reg")[0] + ".nii.gz")
        )

        fig_ori = visualize_t1w(ori_path, show_img=False)
        fig_ham = visualize_t1w(p, show_img=False)

        img_ori = fig_to_ndarray(fig_ori)
        img_ham = fig_to_ndarray(fig_ham)

        plt.close(fig_ori)
        plt.close(fig_ham)

        h1, w1 = img_ori.shape[:2]
        h2, w2 = img_ham.shape[:2]
        width = max(w1, w2)
        height = h1 + h2

        fig_h, fig_w = height / dpi, width / dpi
        fig, axes = plt.subplots(2, 1, figsize=(fig_w, fig_h), dpi=dpi)

        axes[0].imshow(img_ori)
        axes[0].set_title("Original T1-3D", fontsize=9)
        axes[1].imshow(img_ham)
        axes[1].set_title("Harmonized T1-3D", fontsize=9)
        for ax in axes:
            ax.axis("off")

        fig.suptitle(f"sub-{subj}   |   ses-{sess}", fontsize=12, weight="bold")
        plt.tight_layout(rect=[0, 0, 1, 0.94])

        pdf.savefig(fig)
        plt.close(fig)

print(f"All pages written to {pdf_path}")

All pages written to dataset_cache/Harmonization_QC.pdf


In [16]:
def five_slice_lists_center(shape3, step1: int = 5, step2: int = 10, ax_shift: int = 0):
    idx_all = []
    for dim_i, dim in enumerate(shape3):
        c = dim // 2
        offsets = [-step2, -step1, 0, step1, step2]
        if dim_i == 2:
            c += ax_shift
        idx = [min(max(c + off, 0), dim - 1) for off in offsets]
        seen = set()
        idx = [x for x in idx if not (x in seen or seen.add(x))]
        k = 1
        while len(idx) < 5:
            for off in [step2 + k, -(step2 + k)]:
                cand = min(max(c + off, 0), dim - 1)
                if cand not in idx:
                    idx.append(cand)
                    if len(idx) == 5:
                        break
            k += 1
        idx_all.append(np.array(idx[:5]))
    return idx_all


def load_to_ras(nifti_path: Path):
    img = nib.load(str(nifti_path), mmap=True)
    data_native = img.get_fdata(dtype=np.float32)

    src_ornt = io_orientation(img.affine)
    tgt_ornt = axcodes2ornt(("R", "A", "S"))
    transform = ornt_transform(src_ornt, tgt_ornt)
    data_ras = apply_orientation(data_native, transform)
    return data_ras


def extract_slice_ras(img_ras, axis, idx):
    if axis == 0:
        return np.rot90(img_ras[idx, :, :])
    if axis == 1:
        return np.rot90(img_ras[:, idx, :])
    return np.rot90(img_ras[:, :, idx])


root_out = Path(f"{result_dir}/montage_pngs")
root_out.mkdir(exist_ok=True)

for f in df["filepath"]:
    p = Path(f)
    subj, sess = p.parts[6], p.parts[7]
    ori_path = Path(*p.parts[:8]) / "nii" / (p.parts[9].split("_n4_reg")[0] + ".nii.gz")

    img_ori_ras = load_to_ras(ori_path)
    img_ham_ras = load_to_ras(p)

    vmin_o, vmax_o = np.percentile(img_ori_ras, (1, 99))
    vmin_h, vmax_h = np.percentile(img_ham_ras, (1, 99))
    sag_o, cor_o, ax_o = five_slice_lists_center(img_ori_ras.shape, ax_shift=25)
    sag_h, cor_h, ax_h = five_slice_lists_center(img_ham_ras.shape)

    fig, axes = plt.subplots(6, 5, figsize=(11, 13), dpi=100)
    for ax in axes.ravel():
        ax.axis("off")
    for k, idx in enumerate(np.concatenate([sag_o, cor_o, ax_o])):
        axes[k // 5, k % 5].imshow(
            extract_slice_ras(
                img_ori_ras, axis=0 if k < 5 else 1 if k < 10 else 2, idx=idx
            ),
            cmap="gray",
            vmin=vmin_o,
            vmax=vmax_o,
        )
    for k, idx in enumerate(np.concatenate([sag_h, cor_h, ax_h]), start=15):
        r, c = divmod(k, 5)
        axes[r, c].imshow(
            extract_slice_ras(
                img_ham_ras, axis=0 if k < 20 else 1 if k < 25 else 2, idx=idx
            ),
            cmap="gray",
            vmin=vmin_h,
            vmax=vmax_h,
        )
    axes[0, 0].set_title("Original T1 Image", fontsize=12, loc="center")
    axes[3, 0].set_title("Harmonized T1 Image", fontsize=12, loc="center")
    fig.suptitle(f"sub-{subj} | ses-{sess}", fontsize=14, weight="bold")
    plt.tight_layout(rect=[0.03, 0, 0.97, 0.95])
    fig.savefig(root_out / f"sub-{subj}_ses-{sess}.png", dpi=100, bbox_inches="tight")
    plt.close(fig)
    print("Saved", root_out / f"sub-{subj}_ses-{sess}.png")

print("\nAll compare PNGs in", root_out.resolve())

Saved dataset_cache/montage_pngs/sub-BRA012_ses-00.png
Saved dataset_cache/montage_pngs/sub-BRA012_ses-12.png
Saved dataset_cache/montage_pngs/sub-BRA013_ses-00.png
Saved dataset_cache/montage_pngs/sub-BRA013_ses-12.png
Saved dataset_cache/montage_pngs/sub-BRA033_ses-00.png
Saved dataset_cache/montage_pngs/sub-BRA033_ses-12.png
Saved dataset_cache/montage_pngs/sub-BRA041_ses-00.png
Saved dataset_cache/montage_pngs/sub-BRA041_ses-12.png
Saved dataset_cache/montage_pngs/sub-BRA043_ses-00.png
Saved dataset_cache/montage_pngs/sub-BRA043_ses-12.png
Saved dataset_cache/montage_pngs/sub-BRA047_ses-00.png
Saved dataset_cache/montage_pngs/sub-BRA047_ses-12.png
Saved dataset_cache/montage_pngs/sub-BRA051_ses-00.png
Saved dataset_cache/montage_pngs/sub-BRA051_ses-12.png
Saved dataset_cache/montage_pngs/sub-BRA052_ses-00.png
Saved dataset_cache/montage_pngs/sub-BRA052_ses-12.png
Saved dataset_cache/montage_pngs/sub-BRA061_ses-00.png
Saved dataset_cache/montage_pngs/sub-BRA061_ses-12.png
Saved data